In [68]:
!pip install torch
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import PIL

from torch.autograd import Variable

# check the system we're on, gpu
use_cuda = torch.cuda.is_available()

np.load.__defaults__=(None, True, True, 'ASCII')

In [69]:
class HyperParameters():
    def __init__(self):
        self.data = '../Datasets/sketchrnn_chair.npz'
        self.encoder_layer_size = 64
        self.decoder_layer_sze = 128
        # latent vector
        self.Nz = 32
        
        #return
        self.M = 20
        self.dropout = 0.9
        self.batch_size = 100
        self.eta_min = 0.01
        self.R = 0.99995
        self.KL_min = 0.2
        self.wKL = 0.5
        self.lr = 0.001
        self.lr_decay = 0.9999
        self.min_lr = 0.00001
        self.grad_clip = 1.
        self.temperature = 0.4
        self.max_seq_length = 200

hp = HyperParameters()

In [70]:
"""
Padding - each stroke data has different lengths of vectors
- take the largest vector and make all of them the same
"""


'\nPadding - each stroke data has different lengths of vectors\n- take the largest vector and make all of them the same\n'

In [71]:
class DataLoader():
    def __init__(self, file):
        self.file = file
        
    def load_npz(self):
        data = np.load(file=self.file, encoding='latin1', allow_pickle=True)
        return data

dataLoader = DataLoader(hp.data)
data = dataLoader.load_npz()

In [72]:
class DataHandler():
    def __init__(self, data):
        self.data = data
   
    def padding_three_to_five(self):
        max_len = 0
        for datatype in self.data:
            for strokedata in self.data[datatype]:
                print(strokedata.shape)
                if strokedata.shape[0] > max_len:
                    max_len = strokedata.shape[0]
        print(max_len)
        for datatype in self.data:
            for strokedata in self.data[datatype]:
                for i in range(0, strokedata.shape[0]):
                    # extend 3 tuple to 5
                # do padding till max_len
                    
                    

datahandler = DataHandler(data)
datahandler.padding_three_to_five()

(16, 3)
(33, 3)
(24, 3)
(13, 3)
(45, 3)
(34, 3)
(42, 3)
(48, 3)
(28, 3)
(35, 3)
(25, 3)
(47, 3)
(37, 3)
(29, 3)
(35, 3)
(41, 3)
(24, 3)
(38, 3)
(32, 3)
(25, 3)
(23, 3)
(17, 3)
(30, 3)
(20, 3)
(30, 3)
(11, 3)
(46, 3)
(57, 3)
(38, 3)
(32, 3)
(44, 3)
(9, 3)
(54, 3)
(26, 3)
(19, 3)
(8, 3)
(12, 3)
(36, 3)
(45, 3)
(16, 3)
(34, 3)
(10, 3)
(35, 3)
(42, 3)
(46, 3)
(29, 3)
(34, 3)
(29, 3)
(31, 3)
(8, 3)
(51, 3)
(16, 3)
(16, 3)
(18, 3)
(36, 3)
(33, 3)
(40, 3)
(28, 3)
(11, 3)
(42, 3)
(28, 3)
(29, 3)
(37, 3)
(42, 3)
(57, 3)
(9, 3)
(13, 3)
(23, 3)
(17, 3)
(24, 3)
(20, 3)
(13, 3)
(18, 3)
(12, 3)
(12, 3)
(20, 3)
(47, 3)
(8, 3)
(26, 3)
(11, 3)
(34, 3)
(31, 3)
(34, 3)
(36, 3)
(24, 3)
(29, 3)
(22, 3)
(15, 3)
(9, 3)
(11, 3)
(11, 3)
(8, 3)
(9, 3)
(19, 3)
(27, 3)
(30, 3)
(55, 3)
(14, 3)
(13, 3)
(32, 3)
(16, 3)
(36, 3)
(40, 3)
(24, 3)
(21, 3)
(27, 3)
(9, 3)
(35, 3)
(10, 3)
(34, 3)
(35, 3)
(53, 3)
(25, 3)
(23, 3)
(19, 3)
(15, 3)
(30, 3)
(9, 3)
(11, 3)
(42, 3)
(39, 3)
(8, 3)
(11, 3)
(29, 3)
(22, 3)
(8, 3)
(50,

(13, 3)
(51, 3)
(10, 3)
(33, 3)
(20, 3)
(17, 3)
(29, 3)
(10, 3)
(54, 3)
(31, 3)
(11, 3)
(10, 3)
(12, 3)
(36, 3)
(18, 3)
(34, 3)
(8, 3)
(59, 3)
(58, 3)
(15, 3)
(44, 3)
(9, 3)
(38, 3)
(25, 3)
(10, 3)
(49, 3)
(47, 3)
(28, 3)
(13, 3)
(21, 3)
(30, 3)
(15, 3)
(42, 3)
(16, 3)
(26, 3)
(61, 3)
(34, 3)
(39, 3)
(30, 3)
(47, 3)
(14, 3)
(12, 3)
(12, 3)
(31, 3)
(44, 3)
(33, 3)
(13, 3)
(26, 3)
(10, 3)
(31, 3)
(34, 3)
(27, 3)
(33, 3)
(23, 3)
(36, 3)
(29, 3)
(38, 3)
(15, 3)
(20, 3)
(21, 3)
(19, 3)
(31, 3)
(14, 3)
(27, 3)
(16, 3)
(10, 3)
(9, 3)
(46, 3)
(25, 3)
(13, 3)
(34, 3)
(46, 3)
(24, 3)
(27, 3)
(9, 3)
(29, 3)
(37, 3)
(34, 3)
(46, 3)
(11, 3)
(20, 3)
(32, 3)
(39, 3)
(38, 3)
(38, 3)
(41, 3)
(10, 3)
(24, 3)
(27, 3)
(32, 3)
(19, 3)
(10, 3)
(34, 3)
(24, 3)
(17, 3)
(9, 3)
(11, 3)
(17, 3)
(34, 3)
(15, 3)
(16, 3)
(14, 3)
(64, 3)
(34, 3)
(9, 3)
(30, 3)
(31, 3)
(13, 3)
(32, 3)
(14, 3)
(34, 3)
(34, 3)
(20, 3)
(39, 3)
(41, 3)
(42, 3)
(27, 3)
(29, 3)
(35, 3)
(39, 3)
(42, 3)
(22, 3)
(12, 3)
(50, 3)
(34, 3)
(12, 3

(39, 3)
(17, 3)
(20, 3)
(27, 3)
(37, 3)
(18, 3)
(43, 3)
(10, 3)
(30, 3)
(14, 3)
(18, 3)
(34, 3)
(36, 3)
(42, 3)
(27, 3)
(12, 3)
(38, 3)
(26, 3)
(10, 3)
(11, 3)
(10, 3)
(30, 3)
(25, 3)
(10, 3)
(39, 3)
(41, 3)
(18, 3)
(13, 3)
(49, 3)
(32, 3)
(39, 3)
(12, 3)
(39, 3)
(54, 3)
(13, 3)
(36, 3)
(47, 3)
(35, 3)
(20, 3)
(30, 3)
(13, 3)
(24, 3)
(16, 3)
(34, 3)
(33, 3)
(43, 3)
(11, 3)
(50, 3)
(20, 3)
(27, 3)
(30, 3)
(36, 3)
(29, 3)
(66, 3)
(13, 3)
(46, 3)
(30, 3)
(61, 3)
(46, 3)
(30, 3)
(35, 3)
(28, 3)
(48, 3)
(18, 3)
(66, 3)
(17, 3)
(37, 3)
(18, 3)
(14, 3)
(18, 3)
(28, 3)
(11, 3)
(9, 3)
(41, 3)
(37, 3)
(12, 3)
(32, 3)
(17, 3)
(18, 3)
(16, 3)
(21, 3)
(20, 3)
(32, 3)
(42, 3)
(19, 3)
(31, 3)
(58, 3)
(41, 3)
(44, 3)
(17, 3)
(48, 3)
(30, 3)
(20, 3)
(26, 3)
(32, 3)
(23, 3)
(49, 3)
(9, 3)
(29, 3)
(9, 3)
(10, 3)
(12, 3)
(21, 3)
(12, 3)
(36, 3)
(8, 3)
(26, 3)
(9, 3)
(30, 3)
(9, 3)
(31, 3)
(24, 3)
(43, 3)
(33, 3)
(22, 3)
(18, 3)
(16, 3)
(34, 3)
(17, 3)
(49, 3)
(17, 3)
(40, 3)
(15, 3)
(25, 3)
(57, 3)
(60, 3

(31, 3)
(39, 3)
(10, 3)
(46, 3)
(25, 3)
(46, 3)
(49, 3)
(32, 3)
(33, 3)
(29, 3)
(13, 3)
(38, 3)
(20, 3)
(23, 3)
(22, 3)
(34, 3)
(32, 3)
(36, 3)
(38, 3)
(21, 3)
(25, 3)
(8, 3)
(15, 3)
(31, 3)
(31, 3)
(42, 3)
(23, 3)
(51, 3)
(12, 3)
(32, 3)
(51, 3)
(10, 3)
(13, 3)
(39, 3)
(10, 3)
(12, 3)
(30, 3)
(39, 3)
(44, 3)
(12, 3)
(11, 3)
(14, 3)
(11, 3)
(11, 3)
(24, 3)
(44, 3)
(23, 3)
(27, 3)
(23, 3)
(27, 3)
(28, 3)
(11, 3)
(11, 3)
(17, 3)
(13, 3)
(19, 3)
(27, 3)
(61, 3)
(28, 3)
(20, 3)
(50, 3)
(28, 3)
(39, 3)
(46, 3)
(19, 3)
(62, 3)
(28, 3)
(15, 3)
(38, 3)
(37, 3)
(58, 3)
(14, 3)
(8, 3)
(53, 3)
(33, 3)
(12, 3)
(33, 3)
(35, 3)
(12, 3)
(23, 3)
(46, 3)
(10, 3)
(10, 3)
(40, 3)
(16, 3)
(54, 3)
(14, 3)
(20, 3)
(17, 3)
(44, 3)
(13, 3)
(11, 3)
(33, 3)
(48, 3)
(22, 3)
(31, 3)
(32, 3)
(12, 3)
(20, 3)
(13, 3)
(40, 3)
(14, 3)
(40, 3)
(16, 3)
(30, 3)
(32, 3)
(25, 3)
(14, 3)
(35, 3)
(20, 3)
(65, 3)
(50, 3)
(15, 3)
(8, 3)
(9, 3)
(37, 3)
(63, 3)
(38, 3)
(39, 3)
(10, 3)
(52, 3)
(28, 3)
(30, 3)
(25, 3)
(40, 3)
(38,

(31, 3)
(53, 3)
(37, 3)
(32, 3)
(11, 3)
(45, 3)
(39, 3)
(11, 3)
(47, 3)
(30, 3)
(36, 3)
(45, 3)
(24, 3)
(9, 3)
(55, 3)
(11, 3)
(16, 3)
(12, 3)
(55, 3)
(9, 3)
(56, 3)
(12, 3)
(13, 3)
(16, 3)
(37, 3)
(16, 3)
(30, 3)
(20, 3)
(14, 3)
(47, 3)
(10, 3)
(49, 3)
(26, 3)
(38, 3)
(43, 3)
(21, 3)
(16, 3)
(34, 3)
(15, 3)
(17, 3)
(63, 3)
(17, 3)
(46, 3)
(26, 3)
(32, 3)
(25, 3)
(12, 3)
(41, 3)
(33, 3)
(27, 3)
(11, 3)
(28, 3)
(43, 3)
(15, 3)
(13, 3)
(9, 3)
(33, 3)
(30, 3)
(32, 3)
(17, 3)
(20, 3)
(14, 3)
(33, 3)
(12, 3)
(16, 3)
(27, 3)
(14, 3)
(51, 3)
(39, 3)
(24, 3)
(34, 3)
(11, 3)
(11, 3)
(26, 3)
(14, 3)
(27, 3)
(23, 3)
(17, 3)
(11, 3)
(13, 3)
(52, 3)
(19, 3)
(45, 3)
(34, 3)
(32, 3)
(11, 3)
(19, 3)
(14, 3)
(15, 3)
(29, 3)
(8, 3)
(11, 3)
(13, 3)
(18, 3)
(45, 3)
(38, 3)
(11, 3)
(12, 3)
(42, 3)
(38, 3)
(32, 3)
(35, 3)
(16, 3)
(42, 3)
(15, 3)
(41, 3)
(44, 3)
(47, 3)
(46, 3)
(64, 3)
(10, 3)
(31, 3)
(39, 3)
(32, 3)
(29, 3)
(10, 3)
(44, 3)
(30, 3)
(30, 3)
(11, 3)
(10, 3)
(15, 3)
(44, 3)
(41, 3)
(11, 3)
(11,

...

(25, 3)
(8, 3)
(10, 3)
(30, 3)
(29, 3)
(58, 3)
(41, 3)
(14, 3)
(32, 3)
(38, 3)
(16, 3)
(13, 3)
(8, 3)
(20, 3)
(16, 3)
(36, 3)
(17, 3)
(18, 3)
(16, 3)
(46, 3)
(12, 3)
(34, 3)
(33, 3)
(31, 3)
(17, 3)
(21, 3)
(40, 3)
(34, 3)
(37, 3)
(36, 3)
(64, 3)
(11, 3)
(32, 3)
(22, 3)
(60, 3)
(27, 3)
(57, 3)
(35, 3)
(40, 3)
(26, 3)
(27, 3)
(20, 3)
(39, 3)
(28, 3)
(30, 3)
(16, 3)
(15, 3)
(32, 3)
(9, 3)
(12, 3)
(27, 3)
(24, 3)
(20, 3)
(47, 3)
(43, 3)
(50, 3)
(8, 3)
(43, 3)
(34, 3)
(44, 3)
(47, 3)
(32, 3)
(35, 3)
(11, 3)
(21, 3)
(18, 3)
(12, 3)
(30, 3)
(25, 3)
(32, 3)
(34, 3)
(9, 3)
(32, 3)
(33, 3)
(27, 3)
(33, 3)
(10, 3)
(46, 3)
(35, 3)
(29, 3)
(15, 3)
(44, 3)
(42, 3)
(29, 3)
(41, 3)
(38, 3)
(18, 3)
(37, 3)
(19, 3)
(31, 3)
(8, 3)
(12, 3)
(15, 3)
(32, 3)
(16, 3)
(29, 3)
(32, 3)
(17, 3)
(16, 3)
(25, 3)
(27, 3)
(34, 3)
(44, 3)
(54, 3)
(11, 3)
(20, 3)
(22, 3)
(36, 3)
(46, 3)
(9, 3)
(9, 3)
(20, 3)
(13, 3)
(14, 3)
(36, 3)
(63, 3)
(32, 3)
(15, 3)
(41, 3)
(41, 3)
(26, 3)
(38, 3)
(27, 3)
(19, 3)
(30, 3)
(40, 3)


(17, 3)
(41, 3)
(47, 3)
(12, 3)
(27, 3)
(10, 3)
(46, 3)
(36, 3)
(41, 3)
(29, 3)
(42, 3)
(29, 3)
(12, 3)
(13, 3)
(27, 3)
(40, 3)
(15, 3)
(52, 3)
(13, 3)
(35, 3)
(42, 3)
(38, 3)
(45, 3)
(23, 3)
(13, 3)
(44, 3)
(35, 3)
(23, 3)
(22, 3)
(19, 3)
(22, 3)
(11, 3)
(16, 3)
(18, 3)
(10, 3)
(21, 3)
(11, 3)
(57, 3)
(23, 3)
(16, 3)
(49, 3)
(11, 3)
(14, 3)
(14, 3)
(63, 3)
(12, 3)
(25, 3)
(23, 3)
(21, 3)
(34, 3)
(34, 3)
(37, 3)
(19, 3)
(52, 3)
(37, 3)
(36, 3)
(39, 3)
(12, 3)
(17, 3)
(9, 3)
(14, 3)
(16, 3)
(11, 3)
(34, 3)
(45, 3)
(11, 3)
(38, 3)
(24, 3)
(30, 3)
(25, 3)
(19, 3)
(39, 3)
(36, 3)
(10, 3)
(54, 3)
(22, 3)
(16, 3)
(29, 3)
(30, 3)
(14, 3)
(32, 3)
(21, 3)
(44, 3)
(35, 3)
(22, 3)
(28, 3)
(9, 3)
(20, 3)
(15, 3)
(16, 3)
(15, 3)
(27, 3)
(14, 3)
(26, 3)
(11, 3)
(61, 3)
(29, 3)
(25, 3)
(22, 3)
(39, 3)
(35, 3)
(25, 3)
(38, 3)
(17, 3)
(13, 3)
(30, 3)
(39, 3)
(49, 3)
(18, 3)
(17, 3)
(15, 3)
(10, 3)
(9, 3)
(14, 3)
(25, 3)
(11, 3)
(38, 3)
(51, 3)
(9, 3)
(17, 3)
(26, 3)
(28, 3)
(40, 3)
(21, 3)
(8, 3)
(18, 

(27, 3)
(12, 3)
(54, 3)
(22, 3)
(37, 3)
(44, 3)
(16, 3)
(25, 3)
(33, 3)
(27, 3)
(12, 3)
(49, 3)
(16, 3)
(28, 3)
(11, 3)
(8, 3)
(25, 3)
(28, 3)
(20, 3)
(34, 3)
(26, 3)
(9, 3)
(38, 3)
(54, 3)
(24, 3)
(32, 3)
(33, 3)
(57, 3)
(13, 3)
(16, 3)
(25, 3)
(22, 3)
(44, 3)
(27, 3)
(35, 3)
(38, 3)
(21, 3)
(27, 3)
(19, 3)
(14, 3)
(33, 3)
(19, 3)
(32, 3)
(63, 3)
(12, 3)
(53, 3)
(50, 3)
(26, 3)
(10, 3)
(22, 3)
(20, 3)
(64, 3)
(33, 3)
(43, 3)
(10, 3)
(9, 3)
(30, 3)
(40, 3)
(21, 3)
(33, 3)
(12, 3)
(25, 3)
(9, 3)
(11, 3)
(18, 3)
(12, 3)
(45, 3)
(22, 3)
(26, 3)
(44, 3)
(24, 3)
(33, 3)
(15, 3)
(41, 3)
(11, 3)
(13, 3)
(10, 3)
(12, 3)
(34, 3)
(16, 3)
(28, 3)
(48, 3)
(11, 3)
(28, 3)
(22, 3)
(53, 3)
(32, 3)
(35, 3)
(14, 3)
(36, 3)
(10, 3)
(36, 3)
(25, 3)
(12, 3)
(27, 3)
(18, 3)
(21, 3)
(42, 3)
(31, 3)
(24, 3)
(18, 3)
(41, 3)
(18, 3)
(29, 3)
(12, 3)
(28, 3)
(22, 3)
(9, 3)
(31, 3)
(29, 3)
(33, 3)
(45, 3)
(22, 3)
(60, 3)
(15, 3)
(24, 3)
(13, 3)
(15, 3)
(49, 3)
(12, 3)
(12, 3)
(40, 3)
(31, 3)
(18, 3)
(30, 3)
(30, 

(42, 3)
(14, 3)
(18, 3)
(26, 3)
(17, 3)
(16, 3)
(31, 3)
(15, 3)
(41, 3)
(13, 3)
(17, 3)
(59, 3)
(31, 3)
(31, 3)
(18, 3)
(15, 3)
(13, 3)
(29, 3)
(28, 3)
(9, 3)
(17, 3)
(8, 3)
(12, 3)
(51, 3)
(26, 3)
(51, 3)
(33, 3)
(12, 3)
(10, 3)
(22, 3)
(8, 3)
(16, 3)
(23, 3)
(51, 3)
(19, 3)
(16, 3)
(40, 3)
(12, 3)
(15, 3)
(64, 3)
(38, 3)
(45, 3)
(58, 3)
(14, 3)
(11, 3)
(64, 3)
(16, 3)
(9, 3)
(62, 3)
(9, 3)
(11, 3)
(39, 3)
(29, 3)
(18, 3)
(8, 3)
(55, 3)
(15, 3)
(56, 3)
(9, 3)
(56, 3)
(13, 3)
(27, 3)
(25, 3)
(29, 3)
(26, 3)
(19, 3)
(29, 3)
(12, 3)
(12, 3)
(8, 3)
(43, 3)
(17, 3)
(44, 3)
(11, 3)
(27, 3)
(32, 3)
(22, 3)
(27, 3)
(34, 3)
(37, 3)
(10, 3)
(14, 3)
(34, 3)
(35, 3)
(36, 3)
(24, 3)
(32, 3)
(19, 3)
(30, 3)
(49, 3)
(10, 3)
(40, 3)
(27, 3)
(28, 3)
(42, 3)
(13, 3)
(13, 3)
(45, 3)
(8, 3)
(17, 3)
(26, 3)
(35, 3)
(19, 3)
(42, 3)
(28, 3)
(13, 3)
(11, 3)
(12, 3)
(8, 3)
(12, 3)
(46, 3)
(52, 3)
(25, 3)
(15, 3)
(45, 3)
(21, 3)
(30, 3)
(10, 3)
(27, 3)
(16, 3)
(21, 3)
(15, 3)
(19, 3)
(61, 3)
(33, 3)
(17, 3)
(3

(15, 3)
(12, 3)
(45, 3)
(31, 3)
(36, 3)
(34, 3)
(54, 3)
(35, 3)
(45, 3)
(24, 3)
(40, 3)
(28, 3)
(18, 3)
(17, 3)
(19, 3)
(37, 3)
(15, 3)
(24, 3)
(38, 3)
(22, 3)
(31, 3)
(39, 3)
(39, 3)
(19, 3)
(25, 3)
(16, 3)
(16, 3)
(17, 3)
(57, 3)
(24, 3)
(29, 3)
(35, 3)
(10, 3)
(17, 3)
(44, 3)
(39, 3)
(22, 3)
(33, 3)
(40, 3)
(42, 3)
(41, 3)
(22, 3)
(44, 3)
(39, 3)
(23, 3)
(15, 3)
(10, 3)
(31, 3)
(34, 3)
(40, 3)
(17, 3)
(44, 3)
(53, 3)
(40, 3)
(25, 3)
(42, 3)
(14, 3)
(30, 3)
(30, 3)
(11, 3)
(38, 3)
(30, 3)
(40, 3)
(12, 3)
(30, 3)
(36, 3)
(24, 3)
(24, 3)
(18, 3)
(29, 3)
(40, 3)
(24, 3)
(31, 3)
(12, 3)
(25, 3)
(32, 3)
(11, 3)
(48, 3)
(20, 3)
(21, 3)
(25, 3)
(16, 3)
(14, 3)
(11, 3)
(23, 3)
(18, 3)
(22, 3)
(17, 3)
(45, 3)
(14, 3)
(34, 3)
(36, 3)
(40, 3)
(45, 3)
(11, 3)
(31, 3)
(38, 3)
(55, 3)
(47, 3)
(22, 3)
(15, 3)
(13, 3)
(49, 3)
(16, 3)
(11, 3)
(27, 3)
(10, 3)
(34, 3)
(9, 3)
(10, 3)
(29, 3)
(20, 3)
(33, 3)
(10, 3)
(25, 3)
(30, 3)
(46, 3)
(11, 3)
(64, 3)
(9, 3)
(18, 3)
(21, 3)
(26, 3)
(22, 3)
(66, 3)
(1